In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import os
import pickle
import sys

import numpy as np
import pandas as pd
import seaborn as sns
# testing
from pandas.testing import assert_frame_equal
from tqdm import tqdm

import j_utils.munging as mg
from lendingclub.lc_utils import gen_datasets
from lendingclub import config

pd.options.display.max_columns = 999
pd.options.display.max_rows = 60
pd.options.display.max_seq_items = None

# Testing scripts below

In [82]:
from lendingclub.modeling import models

In [83]:
df = pd.read_feather(os.path.join(config.data_dir, 'test_base_loan_info.fth'))
df.shape

(1181, 90)

In [84]:
mpath = config.modeling_dir

In [85]:
ls {mpath}

A_model_dataproc.pkl         C_model_dataproc.pkl  F_model_dataproc.pkl
A_model.pkl                  C_model.pkl           F_model.pkl
baseline_model_dataproc.pkl  D_model_dataproc.pkl  G_model_dataproc.pkl
baseline_model.pkl           D_model.pkl           G_model.pkl
B_model_dataproc.pkl         E_model_dataproc.pkl
B_model.pkl                  E_model.pkl


In [132]:
m = models.Model('B')
m.m

42